# Tuto Neo4j

Le but du tutoriel est de vous faire découvrir Neo4j, une base de données orientée graphe. Nous allons voir comment installer Neo4j, créer une base de données, insérer des données et effectuer des requêtes simples.

On utilisera pour ce tutoriel un container neo4j, ainsi que le dataset suivant : [Social circles: Facebook](https://snap.stanford.edu/data/ego-Facebook.html).

On utilisera les librairies python `py2neo` pour interagir avec Neo4j. Cependant, si l'envie vous vient de faire un projet en Neo4j, utilisez neomodel qui est l'ORM de Neo4j.

### Qu'est ce que Neo4j ?

Neo4j est une base de données orientée graphe qui permet de stocker et de manipuler des données sous forme de graphes. Un graphe est composé de nœuds (ou sommets) et de relations (ou arêtes) entre ces nœuds. Neo4j utilise le langage de requête Cypher pour interagir avec les données.

Une fois que Neo4j est installé, vous pouvez accéder à l'interface web de Neo4j en ouvrant votre navigateur et en allant à l'adresse `http://localhost:7474`. Utilisez les identifiants suivant pour vous connecter : neo4j / strongpassword .

### Création de la base de données

Une fois connecté à l'interface web de Neo4j, vous vous retrouverez sur cette page d'accueil :
![Page d'accueil](./images/screen_1.png "Accueil")

On voit un prompt dans lequel on peut écrire des requêtes Cypher. Commençons par lister les bases de données existantes avec la requête suivante :
```cypher
SHOW DATABASES;
```
Combien de bases de données voyez-vous ? Par défaut, Neo4j crée une base de données nommée `neo4j`. Nous allons créer une nouvelle base de données pour notre tutoriel. Utilisez la requête suivante pour créer une base de données nommée `social` :
```cypher
CREATE DATABASE social;
```

### Insertion des données

On va télécharger les données via un curl, et les charger dans Neo4j avec python.

In [1]:
from pyexpat import features
!wget https://snap.stanford.edu/data/facebook.tar.gz && tar -xvzf facebook.tar.gz && rm facebook.tar.gz

--2025-11-09 16:45:13--  https://snap.stanford.edu/data/facebook.tar.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 732104 (715K) [application/x-gzip]
Saving to: ‘facebook.tar.gz’

facebook.tar.gz     100%[===================>] 714.95K   373KB/s    in 1.9s    

2025-11-09 16:45:15 (373 KB/s) - ‘facebook.tar.gz’ saved [732104/732104]

facebook/
facebook/3980.egofeat
facebook/0.featnames
facebook/698.egofeat
facebook/3437.feat
facebook/3980.featnames
facebook/0.edges
facebook/3437.circles
facebook/686.circles
facebook/348.egofeat
facebook/107.feat
facebook/348.feat
facebook/1912.circles
facebook/3437.egofeat
facebook/698.feat
facebook/348.edges
facebook/1912.feat
facebook/414.circles
facebook/1684.egofeat
facebook/1684.featnames
facebook/1684.feat
facebook/107.egofeat
facebook/0.circles
facebook/414.edges
facebook/698.featnames
fac

On voit que dans le dossier facebook, on a plusieurs fichier avec des extensions différentes :
- `.egofeat` : Caractéristiques des utilisateurs (nœuds)
- `.edges` : Relations entre les utilisateurs (arêtes)
- `.feat` : Caractéristiques des relations (arêtes)
- `.featnames` : Noms des caractéristiques des relations
- `.circles` : Cercles d'amis (groupes d'utilisateurs)


In [11]:
import pandas as pd

extensions = ['egofeat', 'edges', 'feat', 'featnames', 'circles']
edges = pd.read_csv('facebook/107.edges', sep=' ', header=None, names=['source', 'target'])
features = pd.read_csv('facebook/107.feat', sep=' ', header=None)
featnames = pd.read_csv('facebook/107.featnames', sep=' ', header=None)

Combien d'arrêtes et de nœuds y a-t-il dans le graphe ? Combien de caractéristiques par nœud ?

In [16]:
edges.shape, features.shape, featnames.shape

((53498, 2), (1045, 577), (576, 4))

In [3]:
from py2neo import Graph, Node, Relationship

graph = Graph("bolt://neo4j:7687", auth=("neo4j", "strongpassword"), name="social")
graph.run("RETURN 'Connected to Neo4j!' AS message").to_data_frame()

,message
0,Connected to Neo4j!


Ajout des noeuds dans la base de données :

In [32]:
from tqdm import tqdm

for i, row in tqdm(features.iterrows(), total=features.shape[0]):
    node_id = int(row[0])
    node_features = {f"f{j}": int(row[j]) for j in range(1, len(row))}
    person = Node("Person", id=int(node_id), **node_features)
    graph.merge(person, "Person", "id")


  0%|          | 0/1045 [00:00<?, ?it/s]

<class 'int'>


  1%|          | 9/1045 [00:00<01:04, 15.96it/s]

<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>


  3%|▎         | 30/1045 [00:00<00:19, 51.52it/s]

...

<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>


 97%|█████████▋| 1012/1045 [00:06<00:00, 205.92it/s]

<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>


100%|██████████| 1045/1045 [00:06<00:00, 166.61it/s]

<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>


In [35]:
for i, row in tqdm(edges.iterrows(), total=edges.shape[0]):
    src, dst = int(row["source"]), int(row["target"])
    query = """
    MATCH (a:Person {id: $src}), (b:Person {id: $dst})
    MERGE (a)-[:FRIEND_WITH]->(b)
    """
    graph.run(query, src=src, dst=dst)

  6%|▌         | 3310/53498 [00:03<00:51, 976.25it/s] 


KeyboardInterrupt: 

On va maintenant ajouter les cercles d'amis (107.circles), les noms des caractéristiques (107.egofeat), et les caractéristiques des relations (107.featname).

In [36]:
# Ajout des featnames :
for i, row in tqdm(featnames.iterrows(), total=featnames.shape[0]):
    feat_id = int(row[0])
    feat_name = row[1]
    feature = Node("Feature", id=feat_id, name=feat_name)
    graph.merge(feature, "Feature", "id")

100%|██████████| 576/576 [00:01<00:00, 439.51it/s]


Ici on va charger les cercles dans la base de données. Comme les lignes ne sont pas de même longueur, on ne peut pas utiliser pandas.

In [46]:
# ajout des cercles :
circles = []

with open("facebook/107.circles", "r") as f:
    for line in f:
        parts = line.strip().split()  # découpe sur les espaces ou tabulations
        circle_name = parts[0]        # premier élément = nom du cercle
        members = [int(x) for x in parts[1:]]  # reste = membres
        circles.append({"circle": circle_name, "members": members})


for c in circles:
    circle_name = c["circle"]
    for member_id in c["members"]:
        graph.run("""
        MATCH (p:Person {id:$id})
        MERGE (c:Circle {name:$circle})
        MERGE (p)-[:IN_CIRCLE]->(c)
        """, id=member_id, circle=circle_name)

Chargement des caractéristiques des relations (107.egofeat) :

In [47]:
# Chargement de 107.egofeat
with open("facebook/107.egofeat", "r") as f:
    for line in f:
        parts = line.strip().split()
        person_id = int(parts[0])
        features = {f"f{j}": int(parts[j]) for j in range(1, len(parts))}
        query = """
        MATCH (p:Person {id:$id})
        SET p += $features
        """
        graph.run(query, id=person_id, features=features)

Maintenant que les données sont chargées, on peut effectuer des requêtes Cypher pour interroger la base de données. Par exemple, pour trouver tous les amis d'une personne avec l'ID 0, on peut utiliser la requête suivante :

In [48]:
graph.run("""
MATCH (p:Person {id:0})-[:FRIEND_WITH]->(friend)
RETURN friend.id AS friend_id
""").to_data_frame()

,friend_id
0,58
1,171


Maintenant on va lister par ordre décroissant les personnes ayant le plus d'amis :

In [49]:
graph.run("""
MATCH (p:Person)-[:FRIEND_WITH]->(friend)
RETURN p.id AS person_id, COUNT(friend) AS num_friends
ORDER BY num_friends DESC
LIMIT 10
""").to_data_frame()

,person_id,num_friends
0,1888,253
1,1800,244
2,1663,234
3,1352,233
4,1730,225
5,1431,219
6,1199,216
7,1584,210
8,1768,208
9,1589,204


### Visualisation des données dans Neo4j

L'interêt de Neo4j est de pouvoir visualiser les données sous forme de graphe. Pour cela, on peut utiliser l'interface web de Neo4j. Voici comment visualiser les amis de la personne avec l'ID 0 :
```cypher
MATCH (p:Person {id:0})-[:FRIEND_WITH]->(friend)
RETURN p, friend
```
Et voilà le résultat :
![Visualisation des amis](./images/screen_2.png "Amis de la personne 0")

En double cliquant sur le noeud 0, on peut voir les rélations du noeud et ses caractéristiques :
![Détails du noeud](./images/screen_3.png "Détails du noeud 0")

### Fonctions avancées de Cypher

Nous allons utiliser des fonctions plus complèxes de Cypher afin de voir ce que Neo4j apporte de plus par rapport à une base de données relationnelle classique. Pour cela on va utiliser GDS (Graph Data Science) qui est un plugin de Neo4j permettant d'effectuer des analyses de graphes avancées.

Questions :
- Quel est le degré moyen des nœuds dans le graphe ?
- Quelle est la distance moyenne entre deux nœuds dans le graphe ?
- Quels sont les noeuds les plus influents dans le graphe (PageRank) ?
- Quels sont les clusters dans le graphe (algorithme de Louvain) ?

In [7]:
# Degré moyen des nœuds
graph.run("""
CALL gds.graph.project('facebookGraph', 'Person', 'FRIEND_WITH')
YIELD graphName, nodeCount, relationshipCount;
""").to_data_frame()

ClientError: [Procedure.ProcedureCallFailed] Failed to invoke procedure `gds.graph.project`: Caused by: java.lang.IllegalArgumentException: A graph with name 'facebookGraph' already exists.

In [6]:
graph.run("""
CALL gds.degree.stream('facebookGraph')
YIELD nodeId, score
RETURN avg(score) AS average_degree;
""").to_data_frame()

,average_degree
0,51.194258


In [8]:
# Distance moyenne entre deux nœuds
graph.run("""
CALL gds.alpha.allShortestPaths.stream('facebookGraph')
YIELD sourceNodeId, targetNodeId, distance
RETURN avg(distance) AS average_distance;
""").to_data_frame()

,average_distance
0,2.951676


In [11]:
#Quels sont les noeuds les plus influents dans le graphe ?

graph.run("""
CALL gds.pageRank.stream('facebookGraph')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS person_id, score
ORDER BY score DESC
LIMIT 10;
""").to_data_frame()

,person_id,score
0,483,3.634887
1,917,2.956809
2,1888,2.753837
3,1800,2.706385
4,1783,2.621760
5,1352,2.608647
6,1431,2.585110
7,1663,2.584852
8,1086,2.580798
9,1730,2.571580


In [12]:
#Quels sont les clusters dans le graphe (algorithme de Louvain) ?

graph.run("""
CALL gds.louvain.stream('facebookGraph')
YIELD nodeId, communityId
RETURN gds.util.asNode(nodeId).id AS person_id, communityId
ORDER BY communityId
LIMIT 10;
""").to_data_frame()

,person_id,communityId
0,911,15
1,918,22
2,1097,120
3,963,120
4,901,120
5,903,120
6,958,120
7,938,120
8,992,120
9,1118,120
